In [1]:
pip install pandas numpy faker scikit-learn dbldatagen pyspark mysql-connector-python



   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.1/1.8 MB 1.1 MB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.8 MB 1.2 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/1.8 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.8 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.8 MB 1.0 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 930.9 kB/s eta 0:00:02
   ------- -------------------------------- 0.4/1.8 MB 1.0 MB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.8 MB 1.0 MB/s eta 0:00:02
   ----------- ---------------------------- 0.5/1.8 MB 1.1 MB/s eta 0:00:02
   ------------ --------------------------- 0.6/1.8 MB 1.2 MB/s eta 0:00:02
   -------------- ------------------------- 0.7/1.8 MB 1.2 MB/s eta 0:00:01
   -------------

In [4]:
import pandas as pd
import numpy as np
import faker
import random
import mysql.connector
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from pyspark.sql import SparkSession
import time

class DataGenerator:
    def __init__(self, num_customers=1000, num_branches=10, num_transactions=5000):
        self.fake = faker.Faker()
        self.num_customers = num_customers
        self.num_branches = num_branches
        self.num_transactions = num_transactions
        self.spark = SparkSession.builder \
            .appName("DataGeneration") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()

    def format_phone_number(self):
        phone_number = self.fake.phone_number()
        digits = ''.join(filter(str.isdigit, phone_number))
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:10]}"

    def random_date(self, start_date, end_date):
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        return start_dt + timedelta(days=random.randint(0, (end_dt - start_dt).days))

    def generate_customers(self):
        names = [self.fake.name() for _ in range(self.num_customers)]
        first_names = [name.split()[0] for name in names]
        last_names = [name.split()[-1] for name in names]
        addresses = [self.fake.address().replace('\n', ', ') for _ in range(self.num_customers)]
        email_providers = ["gmail.com", "yahoo.com", "outlook.com", "hotmail.com", "aol.com"]
        emails = [f"{first.lower()}.{last.lower()}@{random.choice(email_providers)}" for first, last in zip(first_names, last_names)]
        phone_numbers = [self.format_phone_number() for _ in range(self.num_customers)]
        join_dates = sorted([self.random_date("2017-01-01", "2022-12-31").strftime("%Y-%m-%d") for _ in range(self.num_customers)])
        last_updates = [self.random_date("2023-01-01", "2023-12-31").strftime("%Y-%m-%d %H:%M:%S") for _ in range(self.num_customers)]
        credit_scores = [random.randint(500, 850) for _ in range(self.num_customers)]
        customer_ids = [f"C{index+1000:04d}" for index in range(self.num_customers)]

        customers_df = pd.DataFrame({
            "customer_id": customer_ids,
            "name": names,
            "email": emails,
            "phone": phone_numbers,
            "address": addresses,
            "credit_score": credit_scores,
            "join_date": join_dates,
            "last_update": last_updates
        })

        return self.spark.createDataFrame(customers_df)

    def generate_branches(self):
        branch_ids = [f"B{index:04d}" for index in range(self.num_branches)]
        branch_names = random.choices(["Downtown Branch", "Central Branch", "North Branch", "East Branch", "West Branch"], k=self.num_branches)
        cities = [self.fake.city() for _ in range(100)]
        branch_locations = random.choices(cities, k=self.num_branches)
        branch_timezones = random.choices(["EST", "GMT", "PST", "AEST"], k=self.num_branches)

        branches_df = pd.DataFrame({
            "branch_id": branch_ids,
            "name": branch_names,
            "location": branch_locations,
            "timezone": branch_timezones
        })

        return self.spark.createDataFrame(branches_df)

    def generate_transactions(self, customer_ids, branch_ids):
        transaction_ids = [f"T{index:04d}" for index in range(5000, 10000)]
        X, y = make_regression(n_samples=self.num_transactions, n_features=1, noise=10)
        amounts = np.clip(np.abs(y), 1, 10000)

        num_outliers = int(0.05 * len(amounts))
        outliers_indices = np.random.choice(len(amounts), num_outliers, replace=False)
        amounts[outliers_indices] = np.random.uniform(10000, 100000, size=num_outliers)
        amounts = np.round(amounts, 2)

        status_values = ["completed"] * (int(0.9 * self.num_transactions)) + ["pending"] * int(0.1 * self.num_transactions)
        random.shuffle(status_values)

        random_intervals = np.random.randint(0, 720, size=self.num_transactions)
        cumulative_intervals = np.cumsum(random_intervals)
        start_date = pd.Timestamp("2023-01-01")
        random_timestamps = [start_date + pd.Timedelta(minutes=int(m)) for m in cumulative_intervals]

        transactions_df = pd.DataFrame({
            "transaction_id": transaction_ids,
            "customer_id": random.choices(customer_ids, k=self.num_transactions),
            "branch_id": random.choices(branch_ids, k=self.num_transactions),
            "channel": random.choices(["ATM", "web", "mobile", "branch"], k=self.num_transactions),
            "transaction_type": random.choices(["withdrawal", "deposit", "transfer", "payment"], k=self.num_transactions),
            "amount": amounts,
            "currency": random.choices(["USD", "EUR", "GBP"], k=self.num_transactions),
            "timestamp": random_timestamps,
            "status": status_values
        })

        return self.spark.createDataFrame(transactions_df)

    def load_to_mysql(self, df, table_name):
        db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Harry@1995",
            database="cap_stone"
        )
        cursor = db_connection.cursor()
        for row in df.collect():
            cursor.execute(f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(row))})", tuple(row))
        db_connection.commit()
        cursor.close()
        db_connection.close()

if __name__ == "__main__":
    data_gen = DataGenerator()

    while True:
        customers_spark_df = data_gen.generate_customers()
        branches_spark_df = data_gen.generate_branches()
        transactions_spark_df = data_gen.generate_transactions(
            [row.customer_id for row in customers_spark_df.select("customer_id").collect()],
            [row.branch_id for row in branches_spark_df.select("branch_id").collect()]
        )

        data_gen.load_to_mysql(customers_spark_df, "customers")
        data_gen.load_to_mysql(branches_spark_df, "branches")
        data_gen.load_to_mysql(transactions_spark_df, "transactions")

        print("Data loaded to MySQL successfully!")
        time.sleep(300)  # Wait for 5 minutes


Data loaded to MySQL successfully!


In [2]:
import pandas as pd
import numpy as np
import faker
import random
import mysql.connector
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from pyspark.sql import SparkSession
import time

class DataGenerator:
    def __init__(self, num_customers=1000, num_branches=10):
        self.fake = faker.Faker()
        self.num_customers = num_customers
        self.num_branches = num_branches
        self.spark = SparkSession.builder \
            .appName("DataGeneration") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()

    def format_phone_number(self):
        phone_number = self.fake.phone_number()
        digits = ''.join(filter(str.isdigit, phone_number))
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:10]}"

    def random_date(self, start_date, end_date):
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        return start_dt + timedelta(days=random.randint(0, (end_dt - start_dt).days))

    def generate_customers(self):
        names = [self.fake.name() for _ in range(self.num_customers)]
        first_names = [name.split()[0] for name in names]
        last_names = [name.split()[-1] for name in names]
        addresses = [self.fake.address().replace('\n', ', ') for _ in range(self.num_customers)]
        email_providers = ["gmail.com", "yahoo.com", "outlook.com", "hotmail.com", "aol.com"]
        emails = [f"{first.lower()}.{last.lower()}@{random.choice(email_providers)}" for first, last in zip(first_names, last_names)]
        phone_numbers = [self.format_phone_number() for _ in range(self.num_customers)]
        join_dates = sorted([self.random_date("2017-01-01", "2022-12-31").strftime("%Y-%m-%d") for _ in range(self.num_customers)])
        last_updates = [self.random_date("2023-01-01", "2023-12-31").strftime("%Y-%m-%d %H:%M:%S") for _ in range(self.num_customers)]
        credit_scores = [random.randint(500, 850) for _ in range(self.num_customers)]
        customer_ids = [f"C{index+1000:04d}" for index in range(self.num_customers)]

        customers_df = pd.DataFrame({
            "customer_id": customer_ids,
            "name": names,
            "email": emails,
            "phone": phone_numbers,
            "address": addresses,
            "credit_score": credit_scores,
            "join_date": join_dates,
            "last_update": last_updates
        })

        return self.spark.createDataFrame(customers_df)

    def generate_branches(self):
        branch_ids = [f"B{index:04d}" for index in range(self.num_branches)]
        branch_names = random.choices(["Downtown Branch", "Central Branch", "North Branch", "East Branch", "West Branch"], k=self.num_branches)
        cities = [self.fake.city() for _ in range(100)]
        branch_locations = random.choices(cities, k=self.num_branches)
        branch_timezones = random.choices(["EST", "GMT", "PST", "AEST"], k=self.num_branches)

        branches_df = pd.DataFrame({
            "branch_id": branch_ids,
            "name": branch_names,
            "location": branch_locations,
            "timezone": branch_timezones
        })

        return self.spark.createDataFrame(branches_df)

    def generate_transactions(self, customer_ids, branch_ids, num_transactions=50):
        transaction_ids = [f"T{random.randint(10000, 99999):05d}" for _ in range(num_transactions)]
        X, y = make_regression(n_samples=num_transactions, n_features=1, noise=10)
        amounts = np.clip(np.abs(y), 1, 10000)

        num_outliers = int(0.05 * len(amounts))
        outliers_indices = np.random.choice(len(amounts), num_outliers, replace=False)
        amounts[outliers_indices] = np.random.uniform(10000, 100000, size=num_outliers)
        amounts = np.round(amounts, 2)

        status_values = ["completed"] * (int(0.9 * num_transactions)) + ["pending"] * int(0.1 * num_transactions)
        random.shuffle(status_values)

        random_intervals = np.random.randint(0, 720, size=num_transactions)
        cumulative_intervals = np.cumsum(random_intervals)
        start_date = pd.Timestamp("2023-01-01")
        random_timestamps = [start_date + pd.Timedelta(minutes=int(m)) for m in cumulative_intervals]

        transactions_df = pd.DataFrame({
            "transaction_id": transaction_ids,
            "customer_id": random.choices(customer_ids, k=num_transactions),
            "branch_id": random.choices(branch_ids, k=num_transactions),
            "channel": random.choices(["ATM", "web", "mobile", "branch"], k=num_transactions),
            "transaction_type": random.choices(["withdrawal", "deposit", "transfer", "payment"], k=num_transactions),
            "amount": amounts,
            "currency": random.choices(["USD", "EUR", "GBP"], k=num_transactions),
            "timestamp": random_timestamps,
            "status": status_values
        })

        return self.spark.createDataFrame(transactions_df)

    def load_to_mysql(self, df, table_name):
        db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Harry@1995",
            database="cap_stone"
        )
        cursor = db_connection.cursor()
        for row in df.collect():
            cursor.execute(f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(row))})", tuple(row))
        db_connection.commit()
        cursor.close()
        db_connection.close()

if __name__ == "__main__":
    data_gen = DataGenerator()

    customers_spark_df = data_gen.generate_customers()
    branches_spark_df = data_gen.generate_branches()
    customer_ids = [row.customer_id for row in customers_spark_df.select("customer_id").collect()]
    branch_ids = [row.branch_id for row in branches_spark_df.select("branch_id").collect()]

    data_gen.load_to_mysql(customers_spark_df, "customers")
    data_gen.load_to_mysql(branches_spark_df, "branches")

    while True:
        transactions_spark_df = data_gen.generate_transactions(customer_ids, branch_ids, num_transactions=50)
        data_gen.load_to_mysql(transactions_spark_df, "transactions")

        print("50 transactions appended to MySQL successfully!")
        time.sleep(60)  # Wait for 1 minute


ProgrammingError: 1049 (42000): Unknown database 'cap_stone'

##Final Code

In [1]:
import pandas as pd
import numpy as np
import faker
import random
import mysql.connector
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from pyspark.sql import SparkSession
import time

class DataGenerator:
    def __init__(self, num_customers=50, num_branches=10, num_transactions=150):
        self.fake = faker.Faker()
        self.num_customers = num_customers
        self.num_branches = num_branches
        self.num_transactions = num_transactions
        self.spark = SparkSession.builder \
            .appName("DataGeneration") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()

    def format_phone_number(self):
        phone_number = self.fake.phone_number()
        digits = ''.join(filter(str.isdigit, phone_number))
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:10]}"

    def random_date(self, start_date, end_date):
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        return start_dt + timedelta(days=random.randint(0, (end_dt - start_dt).days))

    def generate_customers(self, customer_id_start):
        names = [self.fake.name() for _ in range(self.num_customers)]
        first_names = [name.split()[0] for name in names]
        last_names = [name.split()[-1] for name in names]
        addresses = [self.fake.address().replace('\n', ', ') for _ in range(self.num_customers)]
        email_providers = ["gmail.com", "yahoo.com", "outlook.com", "hotmail.com", "aol.com"]
        emails = [f"{first.lower()}.{last.lower()}@{random.choice(email_providers)}" for first, last in zip(first_names, last_names)]
        phone_numbers = [self.format_phone_number() for _ in range(self.num_customers)]
        join_dates = sorted([self.random_date("2017-01-01", "2022-12-31").strftime("%Y-%m-%d") for _ in range(self.num_customers)])
        last_updates = [self.random_date("2023-01-01", "2023-12-31").strftime("%Y-%m-%d %H:%M:%S") for _ in range(self.num_customers)]
        credit_scores = [random.randint(500, 850) for _ in range(self.num_customers)]
        customer_ids = [f"C{index + customer_id_start:04d}" for index in range(self.num_customers)]

        customers_df = pd.DataFrame({
            "customer_id": customer_ids,
            "name": names,
            "email": emails,
            "phone": phone_numbers,
            "address": addresses,
            "credit_score": credit_scores,
            "join_date": join_dates,
            "last_update": last_updates,
            "customer_type": [self.fake.job() for _ in range(self.num_customers)]  # Added customer_type
        })

        # Introduce 2% null values
        for column in customers_df.columns:
            customers_df.loc[customers_df.sample(frac=0.02).index, column] = None

        # Introduce 2% duplicates
        duplicates = customers_df.sample(frac=0.02)
        customers_df = pd.concat([customers_df, duplicates], ignore_index=True)

        return self.spark.createDataFrame(customers_df)

    def generate_branches(self):
        branch_ids = [f"B{index:04d}" for index in range(self.num_branches)]
        branch_names = random.choices(["Downtown Branch", "Central Branch", "North Branch", "East Branch", "West Branch"], k=self.num_branches)
        cities = [self.fake.city() for _ in range(100)]
        branch_locations = random.choices(cities, k=self.num_branches)
        branch_timezones = random.choices(["EST", "GMT", "PST", "AEST"], k=self.num_branches)

        branches_df = pd.DataFrame({
            "branch_id": branch_ids,
            "name": branch_names,
            "location": branch_locations,
            "timezone": branch_timezones
        })

        return self.spark.createDataFrame(branches_df)

    def generate_transactions(self, customer_ids, branch_ids, transaction_id_start):
        transaction_ids = [f"T{index + transaction_id_start:04d}" for index in range(self.num_transactions)]
        X, y = make_regression(n_samples=self.num_transactions, n_features=1, noise=10)
        amounts = np.clip(np.abs(y), 1, 10000)

        num_outliers = int(0.05 * len(amounts))
        outliers_indices = np.random.choice(len(amounts), num_outliers, replace=False)
        amounts[outliers_indices] = np.random.uniform(10000, 100000, size=num_outliers)
        amounts = np.round(amounts, 2)

        status_values = ["completed"] * (int(0.9 * self.num_transactions)) + ["pending"] * int(0.1 * self.num_transactions)
        random.shuffle(status_values)

        random_intervals = np.random.randint(0, 720, size=self.num_transactions)
        cumulative_intervals = np.cumsum(random_intervals)
        start_date = pd.Timestamp("2023-01-01")
        random_timestamps = [start_date + pd.Timedelta(minutes=int(m)) for m in cumulative_intervals]

        transactions_df = pd.DataFrame({
            "transaction_id": transaction_ids,
            "customer_id": random.choices(customer_ids, k=self.num_transactions),
            "branch_id": random.choices(branch_ids, k=self.num_transactions),
            "channel": random.choices(["ATM", "web", "mobile", "branch"], k=self.num_transactions),
            "transaction_type": random.choices(["withdrawal", "deposit", "transfer", "payment"], k=self.num_transactions),
            "amount": amounts,
            "currency": random.choices(["USD", "EUR", "GBP"], k=self.num_transactions),
            "timestamp": random_timestamps,
            "status": status_values
        })

        # Introduce 2% null values
        for column in transactions_df.columns:
            transactions_df.loc[transactions_df.sample(frac=0.02).index, column] = None

        # Introduce 2% duplicates
        duplicates = transactions_df.sample(frac=0.02)
        transactions_df = pd.concat([transactions_df, duplicates], ignore_index=True)

        return self.spark.createDataFrame(transactions_df)

    def load_to_mysql(self, df, table_name):
        db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Harry@1995",
            database="cap_stone_new"
        )
        cursor = db_connection.cursor()

        # Prepare INSERT statement with the correct number of placeholders
        columns = df.columns
        num_columns = len(columns)
        placeholders = ', '.join(['%s'] * num_columns)
        insert_stmt = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"

        # Insert rows
        for row in df.collect():
            try:
                cursor.execute(insert_stmt, tuple(row))
            except mysql.connector.Error as err:
                print(f"Error: {err}")

        db_connection.commit()
        cursor.close()
        db_connection.close()

    def get_max_id(self, table_name, id_column):
        db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Harry@1995",
            database="cap_stone_new"
        )
        cursor = db_connection.cursor()
        cursor.execute(f"SELECT MAX({id_column}) FROM {table_name}")
        max_id = cursor.fetchone()[0]
        cursor.close()
        db_connection.close()
        return max_id

    def get_next_customer_id(self):
        max_id = self.get_max_id("customers", "customer_id")
        if max_id is None:
            return 1000
        return int(max_id[1:]) + 1

    def get_next_transaction_id(self):
        max_id = self.get_max_id("transactions", "transaction_id")
        if max_id is None:
            return 5000
        return int(max_id[1:]) + 1

if __name__ == "__main__":
    data_gen = DataGenerator()

    # Generate branches data only if the table is empty
    def is_table_empty(table_name):
        db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Harry@1995",
            database="cap_stone_new"
        )
        cursor = db_connection.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        cursor.close()
        db_connection.close()
        return count == 0

    if is_table_empty("branches"):
        branches_spark_df = data_gen.generate_branches()
        data_gen.load_to_mysql(branches_spark_df, "branches")

    while True:
        customer_id_start = data_gen.get_next_customer_id()
        transaction_id_start = data_gen.get_next_transaction_id()

        customers_spark_df = data_gen.generate_customers(customer_id_start)
        transactions_spark_df = data_gen.generate_transactions(
            [row.customer_id for row in customers_spark_df.select("customer_id").collect()],
            [row.branch_id for row in data_gen.generate_branches().select("branch_id").collect()],
            transaction_id_start
        )

        data_gen.load_to_mysql(customers_spark_df, "customers")
        data_gen.load_to_mysql(transactions_spark_df, "transactions")

        print("Data appended to MySQL successfully!")

        time.sleep(60)  # Wait for 1 minute


Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Data appended to MySQL successfully!
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Data appended to MySQL successfully!
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Data appended to MySQL successfully!
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22): Unknown column 'nan' in 'field list'
Error: 1054 (42S22)